In [53]:
import os
import subprocess
import glob

# POSITIVE MS1 ANNOTATION

In [54]:
# TOOLS PATH
tools_path = "/media/datastorage/it_cast/omnis_microservice_db/tools/thermofisher/"
mzml_dir = "/media/datastorage/it_cast/projects/Radiomica/POS"

raw_files = glob.glob(os.path.join(mzml_dir, "*.raw"))


In [55]:
# thermo file parser
thermo_file_parser = os.path.join(tools_path, "ThermoRawFileParser.exe")

# for raw_file in raw_files:

#     cmd = [
#         "mono", thermo_file_parser, "-i", raw_file, "-o", mzml_dir, "-f", "1"
#     ]
#     try:
#         subprocess.run(cmd, check=True)
#     except subprocess.CalledProcessError as e:
#         print(f"Error processing file {raw_file}: {e}")

In [56]:
# Feature Finder Metabo
mzml_files = glob.glob(os.path.join(mzml_dir, "*.mzML"))
print(mzml_files)
results_dir = os.path.join(mzml_dir, "results")
os.makedirs(results_dir, exist_ok=True)

for mzml_file in mzml_files:
    base_name = os.path.basename(mzml_file).replace(".mzML", "")
    feature_file = os.path.join(results_dir, f"{base_name}.featureXML")

    cmd = [
        "FeatureFinderMetabo",
        "-in", mzml_file,
        "-out", feature_file,
        "-ini", "data/ini/parameters_featurefinder.ini"
    ]
    try:
        subprocess.run(cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing file {mzml_file}: {e}")

['/media/datastorage/it_cast/projects/Radiomica/POS/Sample4_20220312.mzML', '/media/datastorage/it_cast/projects/Radiomica/POS/Sample26_20220312.mzML', '/media/datastorage/it_cast/projects/Radiomica/POS/Sample15_20220312.mzML']
Progress of 'loading spectra list':
-- done [took 2.57 s (CPU), 2.92 s (Wall)] -- 
Progress of 'loading chromatogram list':
-- done [took 0.00 s (CPU), 0.00 s (Wall)] -- 
Progress of 'mass trace detection':
-- done [took 27.65 s (CPU), 28.64 s (Wall)] -- 
Progress of 'elution peak detection':
-- done [took 4.59 s (CPU), 4.72 s (Wall)] -- 
Progress of 'assembling mass traces to features':
Loading metabolite isotope model with 5% RMS error
-- done [took 6.52 s (CPU), 6.75 s (Wall)] -- 
-- FF-Metabo stats --
Input traces:    41571
Output features: 36831 (total trace count: 41571)
Progress of 'Storing featureXML file':
-- done [took 7.07 s (CPU), 7.52 s (Wall)] -- 
FeatureFinderMetabo took 01:08 m (wall), 01:05 m (CPU), 0.90 s (system), 01:04 m (user); Peak Memory U

In [57]:
# Map Aligner 
feature_files = glob.glob(os.path.join(results_dir, "*.featureXML"))
aligned_files = [os.path.join(results_dir, f"aligned_{i + 1}.featureXML") for i in range(len(feature_files))]


cmd = [
    "MapAlignerPoseClustering",
    "-in", *feature_files,
    "-out", *aligned_files,
    "-reference:file", feature_files[0],
    "-ini", "data/ini/parameters_mapaligner.ini",
]
try:
    subprocess.run(cmd, check=True)
except subprocess.CalledProcessError as e:
    print(f"Error during map alignment: {e}")

Progress of 'Aligning input maps':
-- done [took 55.96 s (CPU), 59.14 s (Wall)] -- 
MapAlignerPoseClustering took 01:07 m (wall), 01:03 m (CPU), 1.30 s (system), 01:01 m (user); Peak Memory Usage: 555 MB.


In [58]:
# Feature Linker
output_consensus = os.path.join(results_dir, "consensusXML_file.consensusXML")
cmd = [
    "FeatureLinkerUnlabeledQT",
    "-in", *aligned_files,
    "-out", output_consensus,
    "-ini", "data/ini/parameters_featurelinker.ini",
]
try:
    subprocess.run(cmd, check=True)
except subprocess.CalledProcessError as e:
    print(f"Error during feature linking: {e}")

Linking 3 featureXMLs.
Progress of 'reading input':
-- done [took 19.58 s (CPU), 21.75 s (Wall)] -- 
Progress of 'Linking features':
-- done [took 0.53 s (CPU), 0.65 s (Wall)] -- 
Number of consensus features:
  of size  3:  13844
  of size  2:  17626
  of size  1:  41592
  total:       73062
FeatureLinkerUnlabeledQT took 23.23 s (wall), 20.82 s (CPU), 0.34 s (system), 20.48 s (user); Peak Memory Usage: 555 MB.


In [73]:
# Accurate Mass Search for positive ionization mode
cmd = [
    "AccurateMassSearch",
    "-in", output_consensus,
    "-out", os.path.join(results_dir, "annotated_consensus.mzTab"),
    "-ini", "data/ini/parameters_accuratemass.ini",
    "-db:mapping", "/home/fdotta/dev/omnis/notebooks/HMDBMappingFile.tsv",
    "-db:struct", os.path.join(tools_path, "..", "hmdb_metabolites_structure_2.tsv"),
    "-positive_adducts", "PositiveAdducts.tsv",
    
]

try:
    subprocess.run(cmd, check=True)
except subprocess.CalledProcessError as e:
    print(f"Error during accurate mass search: {e}")

Read 217868 entries from mapping file!
Read 9 entries from adduct file 'PositiveAdducts.tsv'.
Read 15 entries from adduct file '/usr/share/openms/CHEMISTRY/NegativeAdducts.tsv'.
Hits by adduct: #peaks explained (# matching db entries)'
  '2M+3H2O+2H;2+' : 466 (5813)
  '2M+CH3CN+H;1+' : 511 (876)
  '2M+CH3CN+Na;1+' : 588 (1060)
  '2M+K;1+' : 556 (896)
  'M+2H+Na;3+' : 89 (5065)
  'M+3H;3+' : 80 (3697)
  'M+3Na;3+' : 142 (2290)
  'M+H+2Na;3+' : 109 (407)
  'M+H+NH4;2+' : 112 (4658)

AccurateMassSearch took 2.71 s (wall), 2.51 s (CPU), 0.29 s (system), 2.22 s (user); Peak Memory Usage: 1049 MB.


In [74]:

from pyteomics import mztab
import pandas as pd
import numpy as np

# Carica il file mzTab
mztab_file = os.path.join(results_dir, "annotated_consensus.mzTab")
tab_data = mztab.MzTab(mztab_file)

# Per la metabolomica, usa small_molecule_table invece di peptide_table
try:
    # Tabella dei metaboliti (quella che ti serve per metabolomica)
    metabolites_df = pd.DataFrame(tab_data.small_molecule_table)
    print("Small Molecule Table Shape:", metabolites_df.shape)
    print("Columns:", metabolites_df.columns.tolist())
    metabolites_df.head(10)
except AttributeError:
    print("No small molecule table found")

# Verifica anche altre tabelle disponibili
print("\nAvailable tables in mzTab:")
for attr in dir(tab_data):
    if 'table' in attr and not attr.startswith('_'):
        try:
            table = getattr(tab_data, attr)
            if table is not None:
                print(f"- {attr}: {len(table)} entries")
        except:
            pass


Small Molecule Table Shape: (24762, 32)
Columns: ['identifier', 'chemical_formula', 'smiles', 'inchi_key', 'description', 'exp_mass_to_charge', 'calc_mass_to_charge', 'charge', 'retention_time', 'taxid', 'species', 'database', 'database_version', 'spectra_ref', 'search_engine', 'best_search_engine_score[1]', 'search_engine_score[1]_ms_run[1]', 'modifications', 'smallmolecule_abundance_study_variable[1]', 'smallmolecule_abundance_stdev_study_variable[1]', 'smallmolecule_abundance_std_error_study_variable[1]', 'smallmolecule_abundance_study_variable[2]', 'smallmolecule_abundance_stdev_study_variable[2]', 'smallmolecule_abundance_std_error_study_variable[2]', 'smallmolecule_abundance_study_variable[3]', 'smallmolecule_abundance_stdev_study_variable[3]', 'smallmolecule_abundance_std_error_study_variable[3]', 'opt_global_mz_ppm_error', 'opt_global_adduct_ion', 'opt_global_isosim_score', 'opt_global_neutral_mass', 'opt_global_id_group']

Available tables in mzTab:
- peptide_table: 0 entries


In [75]:
def group_annotations_by_id_group(df):
    # Raggruppa i dati basandosi su opt_global_id_group
    grouped = df.groupby('opt_global_id_group').agg({
        'exp_mass_to_charge': 'mean',
        'retention_time': 'mean',
        'description': lambda x: '; '.join(map(str, x.dropna().unique())),  # Convert to string before joining
        'smallmolecule_abundance_study_variable[1]': 'sum',
        'smallmolecule_abundance_study_variable[2]': 'sum',
        'smallmolecule_abundance_study_variable[3]': 'sum',
        
        # Aggiungi altre colonne se necessario
    }).reset_index()

    return grouped

# Esegui il raggruppamento
grouped_df = group_annotations_by_id_group(metabolites_df)
print(grouped_df.head())
print(grouped_df.shape)
grouped_df.to_csv("grouped_annotated_metabolites_radiomica_positive.csv", index=False)

   opt_global_id_group  exp_mass_to_charge  retention_time  \
0                   29          688.523146      909.017765   
1                  106          548.610962       55.897675   
2                  119          153.067109      133.756170   
3                  171          445.184600      497.308735   
4                  178          308.194006      654.216582   

                                         description  \
0  Inositol hexasulfate; 1-(4-Chlorophenyl)-3-((4...   
1  CDP-DG(18:0/PGE2); CDP-DG(20:4(7E,9E,11Z,13E)-...   
2                                     Glycolonitrile   
3                                        Somapacitan   
4  2-(2,4-Dichloro-phenoxy)-N-(2-mercapto-ethyl)-...   

   smallmolecule_abundance_study_variable[1]  \
0                               3.348754e+06   
1                               2.590429e+07   
2                               6.832919e+04   
3                               2.452599e+05   
4                               1.472876e+06   

 

In [ ]:
# # Group annotations (paste after metabolites_df is defined)
# import json
# import numpy as np
# import pandas as pd

# # Column names in your mzTab-derived DataFrame
# MZ_COL = 'exp_mass_to_charge'
# RT_COL = 'retention_time'
# SCORE_COL = 'best_search_engine_score[1]'  # fallback to 'search_engine_score[1]_ms_run[1]' if missing
# NAME_COL = 'description'                   # human-readable name/description
# INCHI_COL = 'inchi_key'
# FORMULA_COL = 'chemical_formula'
# ID_COL = 'identifier'
# SPECTRA_REF_COL = 'spectra_ref'
# CHARGE_COL = 'charge'

# # Tolerance defaults
# DEFAULT_MZ_TOL = '10ppm'  # or numeric Da like 0.01
# DEFAULT_RT_TOL = 10.0      # seconds

# def to_float_safe(x):
#     try:
#         if pd.isna(x):
#             return None
#     except Exception:
#         pass
#     try:
#         return float(x)
#     except Exception:
#         return None

# def mz_tol_da(mz_value, mz_tol):
#     if isinstance(mz_tol, str) and mz_tol.lower().endswith('ppm'):
#         ppm = float(mz_tol[:-3])
#         return mz_value * ppm / 1e6
#     else:
#         return float(mz_tol)

# def group_annotations_mzrt(df, mz_tol=DEFAULT_MZ_TOL, rt_tol=DEFAULT_RT_TOL):
#     df = df.copy().reset_index(drop=True)
#     # Ensure score column exists (choose fallback)
#     score_col = SCORE_COL if SCORE_COL in df.columns else ('search_engine_score[1]_ms_run[1]' if 'search_engine_score[1]_ms_run[1]' in df.columns else None)
#     name_col = NAME_COL if NAME_COL in df.columns else (ID_COL if ID_COL in df.columns else None)

#     # Numeric conversion
#     df['_mz'] = df[MZ_COL].apply(to_float_safe)
#     df['_rt'] = df[RT_COL].apply(to_float_safe)
#     df['_score'] = df[score_col].apply(to_float_safe) if score_col else None

#     # Split rows with and without coords
#     has_coords = df['_mz'].notnull() & df['_rt'].notnull()
#     known = df[has_coords].copy().reset_index(drop=True)
#     unknown = df[~has_coords].copy().reset_index(drop=True)

#     # Sort known and cluster by centroid approach
#     known = known.sort_values(['_mz','_rt']).reset_index(drop=True)
#     clusters = []  # list of dicts {'indices':[ix,...], 'centroid':{'mz':..,'rt':..}}

#     for idx, row in known.iterrows():
#         mz = row['_mz']; rt = row['_rt']
#         placed = False
#         for cl in clusters:
#             mz_cent = cl['centroid']['mz']; rt_cent = cl['centroid']['rt']
#             tol_da = mz_tol_da(mz, mz_tol)
#             if abs(mz - mz_cent) <= tol_da and abs(rt - rt_cent) <= rt_tol:
#                 cl['indices'].append(idx)
#                 n = len(cl['indices'])
#                 # incremental centroid update (mean)
#                 cl['centroid']['mz'] = (mz_cent*(n-1) + mz)/n
#                 cl['centroid']['rt'] = (rt_cent*(n-1) + rt)/n
#                 placed = True
#                 break
#         if not placed:
#             clusters.append({'indices':[idx], 'centroid':{'mz':mz, 'rt':rt}})

#     # Build aggregated rows
#     aggregated = []
#     for cl in clusters:
#         sub = known.loc[cl['indices']]
#         mz_mean = float(np.mean(sub['_mz']))
#         rt_mean = float(np.mean(sub['_rt']))
#         annotations = []
#         for _, r in sub.iterrows():
#             ann = {
#                 'identifier': r.get(ID_COL),
#                 'name': (r.get(name_col) if name_col else None),
#                 'mz': r.get('_mz'),
#                 'rt': r.get('_rt'),
#                 'score': to_float_safe(r.get(score_col)) if score_col else None,
#                 'inchikey': r.get(INCHI_COL) if INCHI_COL in r else None,
#                 'formula': r.get(FORMULA_COL) if FORMULA_COL in r else None,
#                 'charge': r.get(CHARGE_COL) if CHARGE_COL in r else None,
#                 'spectra_ref': r.get(SPECTRA_REF_COL) if SPECTRA_REF_COL in r else None
#             }
#             annotations.append({k:v for k,v in ann.items() if v is not None})
#         # top candidate by score if available
#         top_score = None; top_name = None
#         if score_col:
#             scored = [(to_float_safe(a.get('score')), a.get('name') or a.get('identifier')) for a in annotations if a.get('score') is not None]
#             if scored:
#                 best = max(scored, key=lambda x: x[0])
#                 top_score, top_name = best[0], best[1]
#         inchikeys = list({a.get('inchikey') for a in annotations if a.get('inchikey')})
#         formulas = list({a.get('formula') for a in annotations if a.get('formula')})
#         aggregated.append({
#             'mz_mean': mz_mean,
#             'rt_mean': rt_mean,
#             'n_annotations': len(annotations),
#             'annotations': json.dumps(annotations, default=str),
#             'top_name': top_name,
#             'top_score': top_score,
#             'inchikeys': ';'.join(inchikeys) if inchikeys else None,
#             'formulas': ';'.join(formulas) if formulas else None
#         })

#     agg_df = pd.DataFrame(aggregated)

#     # Add unknown rows as singletons with available fields
#     if len(unknown):
#         unknown_rows = []
#         for _, r in unknown.iterrows():
#             ann = {
#                 'identifier': r.get(ID_COL),
#                 'name': r.get(name_col) if name_col else None,
#                 'mz': to_float_safe(r.get(MZ_COL)),
#                 'rt': to_float_safe(r.get(RT_COL)),
#                 'score': to_float_safe(r.get(score_col)) if score_col else None,
#                 'inchikey': r.get(INCHI_COL) if INCHI_COL in r else None,
#                 'formula': r.get(FORMULA_COL) if FORMULA_COL in r else None,
#                 'charge': r.get(CHARGE_COL) if CHARGE_COL in r else None,
#                 'spectra_ref': r.get(SPECTRA_REF_COL) if SPECTRA_REF_COL in r else None
#             }
#             unknown_rows.append({
#                 'mz_mean': ann['mz'],
#                 'rt_mean': ann['rt'],
#                 'n_annotations': 1,
#                 'annotations': json.dumps([ann], default=str),
#                 'top_name': ann.get('name'),
#                 'top_score': ann.get('score'),
#                 'inchikeys': ann.get('inchikey'),
#                 'formulas': ann.get('formula')
#             })
#         agg_df = pd.concat([agg_df, pd.DataFrame(unknown_rows)], ignore_index=True, sort=False)

#     # sort and return
#     agg_df = agg_df.sort_values(['rt_mean','mz_mean']).reset_index(drop=True)
#     return agg_df

# # Run grouping
# print("Input rows:", len(metabolites_df))
# grouped = group_annotations_mzrt(metabolites_df, mz_tol='10ppm', rt_tol=5.0)
# print("Grouped rows:", len(grouped))
# # preview
# display(grouped.head(20))

# # Save to CSV
# grouped.to_csv(os.path.join(results_dir if 'results_dir' in globals() else '.', "annotated_metabolites_grouped.csv"), index=False)
# print("Saved annotated_metabolites_grouped_radiomica.csv")

Input rows: 564629
Grouped rows: 39042


,mz_mean,rt_mean,n_annotations,annotations,top_name,top_score,inchikeys,formulas
0,197.963336,-22.586928,1,"[{""identifier"": ""HMDB:HMDB0003585"", ""name"": ""T...",None,None,"InChI=1S/C3H7NO2S2/c4-2(1-8-7)3(5)6/h2,7H,1,4H...",C3H7NO2S2
1,255.822029,-22.586928,1,"[{""identifier"": ""HMDB:HMDB0039242"", ""name"": ""L...",None,None,InChI=1S/C41H68O10/c1-21(2)22-11-14-38(5)17-18...,C41H68O10
2,436.876652,-22.586928,7,"[{""identifier"": ""HMDB:HMDB0042069"", ""name"": ""T...",None,None,InChI=1S/C47H88O6/c1-4-7-10-13-16-19-22-23-26-...,C47H88O6
3,114.951616,-22.113280,2,"[{""identifier"": ""HMDB:HMDB0000257"", ""name"": ""T...",None,None,"InChI=1S/H2O3S2/c1-5(2,3)4/h(H2,1,2,3,4)",H2O3S2
4,255.087319,-22.113280,1,"[{""identifier"": ""HMDB:HMDB0015656"", ""name"": ""N...",None,None,InChI=1S/C14H14N2/c1-2-7-13-11(4-1)5-3-6-12(13...,C14H14N2
5,173.045663,-22.113280,13,"[{""identifier"": ""HMDB:HMDB0029905"", ""name"": ""A...",None,None,"InChI=1S/C4H6S/c1-3-5-4-2/h3-4H,1-2H2;InChI=1S...",C12H10N2OS2;C15H11O7;C26H26O7;C4H6S;C18H14N2O4
6,145.014342,-21.639101,3,"[{""identifier"": ""HMDB:HMDB0032429"", ""name"": ""3...",None,None,"InChI=1S/C6H8S2/c1-2-6-7-4-3-5-8-6/h2-4,6H,1,5...",C6H8S2
7,328.860405,-21.639101,2,"[{""identifier"": ""HMDB:HMDB0112880"", ""name"": ""P...",None,None,InChI=1S/C52H88NO10P/c1-3-5-7-9-11-13-15-17-19...,C52H88NO10P
8,131.035055,-20.689248,3,"[{""identifier"": ""HMDB:HMDB0034692"", ""name"": ""9...",None,None,InChI=1S/C11H14N2O3/c12-9(11(16)13-7-10(14)15)...,C11H14N2O3;C22H18O4
9,150.056158,-20.215070,11,"[{""identifier"": ""HMDB:HMDB0041037"", ""name"": ""8...",None,None,InChI=1S/C16H20O3/c1-15(2)13(18-15)8-10-16(3)9...,C15H19ClO4;C16H20O3;C14H22O2S


Saved annotated_metabolites_grouped_radiomica.csv


In [76]:
## positive mode example
# TOOLS PATH
tools_path = "/media/datastorage/it_cast/omnis_microservice_db/tools/thermofisher/"
mzml_dir = "/media/datastorage/it_cast/projects/Radiomica/NEG"


raw_files = glob.glob(os.path.join(mzml_dir, "*.raw"))
print(raw_files)

['/media/datastorage/it_cast/projects/Radiomica/NEG/Sample15_20220315.raw', '/media/datastorage/it_cast/projects/Radiomica/NEG/Sample4_20220315.raw', '/media/datastorage/it_cast/projects/Radiomica/NEG/Sample26_20220315.raw']


In [77]:
# thermo file parser
thermo_file_parser = os.path.join(tools_path, "ThermoRawFileParser.exe")

# for raw_file in raw_files:

#     cmd = [
#         "mono", thermo_file_parser, "-i", raw_file, "-o", mzml_dir, "-f", "1"
#     ]
#     try:
#         subprocess.run(cmd, check=True)
#     except subprocess.CalledProcessError as e:
#         print(f"Error processing file {raw_file}: {e}")

In [78]:
# Feature Finder Metabo
mzml_files = glob.glob(os.path.join(mzml_dir, "*.mzML"))
print(mzml_files)
results_dir = os.path.join(mzml_dir, "results")
os.makedirs(results_dir, exist_ok=True)

for mzml_file in mzml_files:
    base_name = os.path.basename(mzml_file).replace(".mzML", "")
    feature_file = os.path.join(results_dir, f"{base_name}.featureXML")

    cmd = [
        "FeatureFinderMetabo",
        "-in", mzml_file,
        "-out", feature_file,
        "-ini", "data/ini/parameters_featurefinder.ini",
    ]
    try:
        subprocess.run(cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing file {mzml_file}: {e}")

['/media/datastorage/it_cast/projects/Radiomica/NEG/Sample4_20220315.mzML', '/media/datastorage/it_cast/projects/Radiomica/NEG/Sample26_20220315.mzML', '/media/datastorage/it_cast/projects/Radiomica/NEG/Sample15_20220315.mzML']
Progress of 'loading spectra list':
-- done [took 2.69 s (CPU), 2.83 s (Wall)] -- 
Progress of 'loading chromatogram list':
-- done [took 0.00 s (CPU), 0.00 s (Wall)] -- 
Progress of 'mass trace detection':
-- done [took 22.67 s (CPU), 23.51 s (Wall)] -- 
Progress of 'elution peak detection':
-- done [took 4.52 s (CPU), 4.69 s (Wall)] -- 
Progress of 'assembling mass traces to features':
Loading metabolite isotope model with 5% RMS error
-- done [took 5.50 s (CPU), 5.67 s (Wall)] -- 
-- FF-Metabo stats --
Input traces:    35998
Output features: 31806 (total trace count: 35998)
Progress of 'Storing featureXML file':
-- done [took 12.81 s (CPU), 13.45 s (Wall)] -- 
FeatureFinderMetabo took 01:09 m (wall), 01:06 m (CPU), 1.17 s (system), 01:05 m (user); Peak Memory

In [79]:
# Map Aligner 
feature_files = glob.glob(os.path.join(results_dir, "*.featureXML"))
print(feature_files)
aligned_files = [os.path.join(results_dir, f"aligned_{i + 1}.featureXML") for i in range(len(feature_files))]
print(aligned_files)

cmd = [
    "MapAlignerPoseClustering",
    "-in", *feature_files,
    "-out", *aligned_files,
    "-reference:file", feature_files[0],
    "-ini", "data/ini/parameters_mapaligner.ini",
]
try:
    subprocess.run(cmd, check=True)
except subprocess.CalledProcessError as e:
    print(f"Error during map alignment: {e}")

['/media/datastorage/it_cast/projects/Radiomica/NEG/results/Sample15_20220315.featureXML', '/media/datastorage/it_cast/projects/Radiomica/NEG/results/Sample26_20220315.featureXML', '/media/datastorage/it_cast/projects/Radiomica/NEG/results/Sample4_20220315.featureXML']
['/media/datastorage/it_cast/projects/Radiomica/NEG/results/aligned_1.featureXML', '/media/datastorage/it_cast/projects/Radiomica/NEG/results/aligned_2.featureXML', '/media/datastorage/it_cast/projects/Radiomica/NEG/results/aligned_3.featureXML']
Progress of 'Aligning input maps':
-- done [took 01:23 m (CPU), 01:29 m (Wall)] -- 
MapAlignerPoseClustering took 01:37 m (wall), 01:31 m (CPU), 1.39 s (system), 01:30 m (user); Peak Memory Usage: 1046 MB.


In [81]:
# Feature Linker
output_consensus = os.path.join(results_dir, "consensusXML_file.consensusXML")
cmd = [
    "FeatureLinkerUnlabeledQT",
    "-in", *aligned_files,
    "-out", output_consensus,
    "-ini", "data/ini/parameters_featurelinker.ini",
]
try:
    subprocess.run(cmd, check=True)
except subprocess.CalledProcessError as e:
    print(f"Error during feature linking: {e}")

Linking 3 featureXMLs.
Progress of 'reading input':
-- done [took 28.06 s (CPU), 29.23 s (Wall)] -- 
Progress of 'Linking features':
-- done [took 0.41 s (CPU), 0.45 s (Wall)] -- 
Number of consensus features:
  of size  3:  17331
  of size  2:  11319
  of size  1:  21430
  total:       50080
FeatureLinkerUnlabeledQT took 30.85 s (wall), 29.00 s (CPU), 0.44 s (system), 28.56 s (user); Peak Memory Usage: 1046 MB.


In [82]:
# Accurate Mass Search
cmd = [
    "AccurateMassSearch",
    "-in", output_consensus,
    "-out", os.path.join(results_dir, "annotated_consensus.mzTab"),
    "-ini", "data/ini/parameters_accuratemass_negative.ini",
    "-db:mapping", "/home/fdotta/dev/omnis/notebooks/HMDBMappingFile.tsv",
    "-db:struct", os.path.join(tools_path, "..", "hmdb_metabolites_structure_2.tsv"),
]

try:
    subprocess.run(cmd, check=True)
except subprocess.CalledProcessError as e:
    print(f"Error during accurate mass search: {e}")

Read 217868 entries from mapping file!
Read 33 entries from adduct file '/usr/share/openms/CHEMISTRY/PositiveAdducts.tsv'.
Read 15 entries from adduct file '/usr/share/openms/CHEMISTRY/NegativeAdducts.tsv'.
Hits by adduct: #peaks explained (# matching db entries)'
  '2M+C2H4O2-H;1-' : 224 (308)
  '2M+CH2O2-H;1-' : 215 (363)
  '2M-H;1-' : 151 (279)
  '3M-H;1-' : 143 (222)
  'M+Br;1-' : 352 (843)
  'M+C2H4O2-H;1-' : 150 (1697)
  'M+C2HO2F3-H;1-' : 276 (1839)
  'M+CH2O2-H;1-' : 145 (240)
  'M+Cl;1-' : 167 (419)
  'M+K-2H;1-' : 202 (515)
  'M+Na-2H;1-' : 106 (392)
  'M-2H;2-' : 93 (367)
  'M-3H;3-' : 106 (535)
  'M-H2O-H;1-' : 76 (118)
  'M-H;1-' : 93 (282)

AccurateMassSearch took 1.84 s (wall), 1.67 s (CPU), 0.13 s (system), 1.54 s (user); Peak Memory Usage: 1046 MB.


In [83]:
from pyteomics import mztab
import pandas as pd
import numpy as np

# Carica il file mzTab
mztab_file = os.path.join(results_dir, "annotated_consensus.mzTab")
tab_data = mztab.MzTab(mztab_file)

# Per la metabolomica, usa small_molecule_table invece di peptide_table
try:
    # Tabella dei metaboliti (quella che ti serve per metabolomica)
    metabolites_df = pd.DataFrame(tab_data.small_molecule_table)
    print("Small Molecule Table Shape:", metabolites_df.shape)
    print("Columns:", metabolites_df.columns.tolist())
    metabolites_df.head(10)
except AttributeError:
    print("No small molecule table found")

# Verifica anche altre tabelle disponibili
print("\nAvailable tables in mzTab:")
for attr in dir(tab_data):
    if 'table' in attr and not attr.startswith('_'):
        try:
            table = getattr(tab_data, attr)
            if table is not None:
                print(f"- {attr}: {len(table)} entries")
        except:
            pass


# SALVA IL CSV totale
print(metabolites_df.columns)
metabolites_df.to_csv("negative_annotated_metabolites_radiomica.csv", index=False)  

Small Molecule Table Shape: (8419, 32)
Columns: ['identifier', 'chemical_formula', 'smiles', 'inchi_key', 'description', 'exp_mass_to_charge', 'calc_mass_to_charge', 'charge', 'retention_time', 'taxid', 'species', 'database', 'database_version', 'spectra_ref', 'search_engine', 'best_search_engine_score[1]', 'search_engine_score[1]_ms_run[1]', 'modifications', 'smallmolecule_abundance_study_variable[1]', 'smallmolecule_abundance_stdev_study_variable[1]', 'smallmolecule_abundance_std_error_study_variable[1]', 'smallmolecule_abundance_study_variable[2]', 'smallmolecule_abundance_stdev_study_variable[2]', 'smallmolecule_abundance_std_error_study_variable[2]', 'smallmolecule_abundance_study_variable[3]', 'smallmolecule_abundance_stdev_study_variable[3]', 'smallmolecule_abundance_std_error_study_variable[3]', 'opt_global_mz_ppm_error', 'opt_global_adduct_ion', 'opt_global_isosim_score', 'opt_global_neutral_mass', 'opt_global_id_group']

Available tables in mzTab:
- peptide_table: 0 entries
-

In [84]:
def group_annotations_by_id_group(df):
    # Raggruppa i dati basandosi su opt_global_id_group
    grouped = df.groupby('opt_global_id_group').agg({
        'exp_mass_to_charge': 'mean',
        'retention_time': 'mean',
        'description': lambda x: '; '.join(map(str, x.dropna().unique())),  # Convert to string before joining
        'smallmolecule_abundance_study_variable[1]': 'sum',
        'smallmolecule_abundance_study_variable[2]': 'sum',
        'smallmolecule_abundance_study_variable[3]': 'sum',
        # Aggiungi altre colonne se necessario
    }).reset_index()

    return grouped

# Esegui il raggruppamento
grouped_df = group_annotations_by_id_group(metabolites_df)
print(grouped_df.head())

   opt_global_id_group  exp_mass_to_charge  retention_time  \
0                  124          399.347804     1169.817936   
1                  149          419.134358      576.982773   
2                  195          327.253836      855.717222   
3                  221          422.181752      467.905584   
4                  238          248.121924     1013.370875   

                                         description  \
0  3,4-DHPEA-EA; 2-(3,4-Dihydroxyphenylethyl)-6-e...   
1                  4,5-Dichloro-3H-1,3-dithiol-2-one   
2                                  Taurine glutamate   
3      O-Phosphoethanolamine; Aminoethanol phosphate   
4                                 Pyridoxaminium(1+)   

   smallmolecule_abundance_study_variable[1]  \
0                               2.853603e+05   
1                               5.493803e+04   
2                               1.638944e+07   
3                               1.370257e+05   
4                               2.586636e+05   

 

In [85]:
grouped_df.to_csv("negative_annotated_metabolites_grouped.csv", index=False)

In [86]:
grouped_df.shape

(2394, 7)